In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
from matplotlib.dates import ConciseDateFormatter, AutoDateLocator
%matplotlib inline
import seaborn as sns

from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

FIGSIZE = (28,10)

In [ ]:
sales = pd.read_csv('./data/electronic_sales.csv')

In [ ]:
month_names = [month for month in sales.columns if month != 'YEAR']
sales_series_list = []
for idx, month in enumerate(month_names):
    month_num = idx+1
    month_df = sales[['YEAR', month]].copy()
    month_df['DATE'] = month_df.YEAR.apply(lambda y: f"{y}-{month_num}-01")
    month_df['DATE'] = pd.to_datetime(month_df.DATE)
    month_df = month_df.set_index('DATE')
    month_df = month_df.drop(columns=['YEAR'])
    month_df = month_df.rename(columns={month: 'SALES'})
    sales_series_list.append(month_df)
monthly_sales = pd.concat(sales_series_list).sort_index().dropna().asfreq('MS')
train_test_split_date = '2024-01-01'
monthly_sales_train = monthly_sales[monthly_sales.index < train_test_split_date]
monthly_sales_test = monthly_sales[monthly_sales.index >= train_test_split_date]

In [ ]:
plt.figure(figsize=FIGSIZE)
plt.xticks(rotation=90)
plt.xlabel("Year")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title("U.S. Census Bureau Total sales estimates (millions of dollars)")
plt.suptitle("Electronics & appliance stores")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales, x=monthly_sales.index, y=monthly_sales.SALES, linewidth=1.5)

In [ ]:
monthly_sales_decade_MV = monthly_sales.SALES.rolling(window=120, center=True, min_periods=60).mean()
monthly_sales_month_MV = monthly_sales.SALES.rolling(window=12, center=True, min_periods=6).mean()

In [ ]:
plt.figure(figsize=FIGSIZE)
plt.xticks(rotation=90)
plt.xlabel("Year")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title("U.S. Census Bureau Total sales estimates (millions of dollars)")
plt.suptitle("Electronics & appliance stores")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales, x=monthly_sales.index, y=monthly_sales.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=monthly_sales_decade_MV.index, y=monthly_sales_decade_MV, linewidth=3, label="Decade MA")
G = sns.lineplot(ax=ax, x=monthly_sales_month_MV.index, y=monthly_sales_month_MV, linewidth=3, label="Month MA")

In [ ]:
trend_model_DP = DeterministicProcess(index=monthly_sales_train.index, constant=True, order=1, drop=True)
trend_model_X_train = trend_model_DP.in_sample()
trend_model_y_train = monthly_sales_train.SALES
trend_model_X_test = trend_model_DP.out_of_sample(steps=len(monthly_sales_test))
trend_model_y_test = monthly_sales_test.SALES
trend_model = LinearRegression(fit_intercept=False)

trend_model.fit(trend_model_X_train, trend_model_y_train)
trend_model_fit = pd.Series(trend_model.predict(trend_model_X_train), index=trend_model_X_train.index)
trend_model_preds = pd.Series(trend_model.predict(trend_model_X_test), index=trend_model_X_test.index)
trend_model_RMSE = root_mean_squared_error(trend_model_y_test, trend_model_preds)

In [ ]:
plt.figure(figsize=FIGSIZE)
plt.xticks(rotation=90)
plt.xlabel("Year")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title(f"Trend Model Predictions (RMSE: {trend_model_RMSE:,.2F})")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales, x=monthly_sales.index, y=monthly_sales.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=trend_model_fit.index, y=trend_model_fit, linewidth=3, label="Model Fit")
G = sns.lineplot(ax=ax, x=trend_model_preds.index, y=trend_model_preds, linewidth=3, label="Model Predictions")

In [ ]:
plt.xticks(rotation=90)
plt.xlabel("Month")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title(f"Linear Model Predictions Vs Actual Sales (2024)")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales_test, x=monthly_sales_test.index, y=monthly_sales_test.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=trend_model_preds.index, y=trend_model_preds, linewidth=3, label="Model Predictions")

In [ ]:
def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("365D") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [ ]:
plot_periodogram(monthly_sales.SALES);

In [ ]:
seasonal_dp = DeterministicProcess(index=monthly_sales_train.index, constant=True, order=4, drop=True)
seasonal_X_train = seasonal_dp.in_sample()
seasonal_y_train = monthly_sales_train.SALES
seasonal_X_valid = seasonal_dp.out_of_sample(steps=len(monthly_sales_test))
seasonal_y_valid = monthly_sales_test.SALES

seasonal_model = LinearRegression(fit_intercept=False)
seasonal_model.fit(seasonal_X_train, seasonal_y_train)
seasonal_model_fit = pd.Series(seasonal_model.predict(seasonal_X_train), index=seasonal_X_train.index)
seasonal_model_preds = pd.Series(seasonal_model.predict(seasonal_X_valid), index=seasonal_X_valid.index)
season_model_RMSE = root_mean_squared_error(seasonal_y_valid, seasonal_model_preds)

In [ ]:
plt.figure(figsize=FIGSIZE)
plt.xticks(rotation=90)
plt.xlabel("Year")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title(f"Quadratic Model Predictions (RMSE: {season_model_RMSE:,.2F})")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales, x=monthly_sales.index, y=monthly_sales.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=seasonal_model_fit.index, y=seasonal_model_fit, linewidth=3, label="Model Fit")
G = sns.lineplot(ax=ax, x=seasonal_model_preds.index, y=seasonal_model_preds, linewidth=3, label="Model Predictions")

In [ ]:
plt.xticks(rotation=90)
plt.xlabel("Month")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title(f"Seasonal Model Predictions Vs Actual Sales (2024)")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales_test, x=monthly_sales_test.index, y=monthly_sales_test.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=seasonal_model_preds.index, y=seasonal_model_preds, linewidth=3, label="Model Predictions")

In [ ]:
def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax

def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig

In [ ]:
_ = plot_lags(monthly_sales.SALES, lags=12, nrows=2)

In [ ]:
_ = plot_pacf(monthly_sales.SALES, lags=12)

In [ ]:
lagged_X_train = pd.DataFrame({'lag_1': monthly_sales_train.SALES.shift(1), 'lag_3': monthly_sales_train.SALES.shift(3)}, index=monthly_sales_train.index).dropna()
lagged_y_train = lagged_X_train.join(monthly_sales_train).SALES
monthly_sales_test_lagged_data = monthly_sales[monthly_sales.index >= '2023-10-01']
lagged_X_valid = pd.DataFrame({'lag_1': monthly_sales_test_lagged_data.SALES.shift(1), 'lag_3': monthly_sales_test_lagged_data.SALES.shift(3)}, index=monthly_sales_test_lagged_data.index).dropna()
ladded_y_valid = lagged_X_valid.join(monthly_sales_test_lagged_data).SALES
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(lagged_X_train, lagged_y_train, test_size=0.2, random_state=1)

# lagged_model = LinearRegression()
# best_rate = 0.01
# best_RMSE = 10000000000
# for i in range(1, 30):
#     test_rate = i / 100
#     lagged_model = XGBRegressor(random_state=1, n_estimators=2000, learning_rate=test_rate, early_stopping_rounds=10)
#     lagged_model.fit(lagged_X_train, lagged_y_train, eval_set=[(X_val_split, y_val_split)], verbose=False)
#     lagged_model_fit = pd.Series(lagged_model.predict(lagged_X_train), index=lagged_X_train.index)
#     lagged_model_preds = pd.Series(lagged_model.predict(lagged_X_valid), index=lagged_X_valid.index)
#     lagged_model_RMSE = root_mean_squared_error(ladded_y_valid, lagged_model_preds)
#     print(test_rate, lagged_model_RMSE)
#     if lagged_model_RMSE < best_RMSE:
#         best_RMSE = lagged_model_RMSE
#         best_rate = test_rate
# print()
# print('Best Rate: ', best_rate)
# print('Best RMSE: ', best_RMSE)

lagged_model = XGBRegressor(random_state=1, n_estimators=2000, learning_rate=0.18, early_stopping_rounds=10)
lagged_model.fit(lagged_X_train, lagged_y_train, eval_set=[(X_val_split, y_val_split)], verbose=False)
lagged_model_fit = pd.Series(lagged_model.predict(lagged_X_train), index=lagged_X_train.index)
lagged_model_preds = pd.Series(lagged_model.predict(lagged_X_valid), index=lagged_X_valid.index)
lagged_model_RMSE = root_mean_squared_error(ladded_y_valid, lagged_model_preds)
lagged_model_MAPE = mean_absolute_percentage_error(ladded_y_valid, lagged_model_preds)

In [ ]:
plt.figure(figsize=FIGSIZE)
plt.xticks(rotation=90)
plt.xlabel("Year")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title(f"Lagged Model Predictions (RMSE: {lagged_model_RMSE:,.2F})")

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales, x=monthly_sales.index, y=monthly_sales.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=lagged_model_fit.index, y=lagged_model_fit, linewidth=3, label="Model Fit")
G = sns.lineplot(ax=ax, x=lagged_model_preds.index, y=lagged_model_preds, linewidth=3, label="Model Predictions")

In [ ]:
plt.xticks(rotation=90)
plt.xlabel("Month")
plt.ylabel("Total sales estimates (millions of dollars)")
plt.title(f"Lagged Model Predictions Vs Actual Sales (2024)")
plt.ylim(0, 9000)

ax = plt.gca()
locator = AutoDateLocator()
formatter = ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

G = sns.lineplot(ax=ax, data=monthly_sales_test, x=monthly_sales_test.index, y=monthly_sales_test.SALES, linewidth=1.5, alpha=0.2)
G = sns.lineplot(ax=ax, x=lagged_model_preds.index, y=lagged_model_preds, linewidth=3, label="Model Predictions")

In [ ]:
print(f"{'Actual on April, 2024:':30} {monthly_sales.loc['2024-04-01'].SALES:.2f}")
print(f"{'Predicted on April, 2024:':30} {lagged_model_preds.loc['2024-04-01']:.2f}")